In [55]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.|
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
print(os.listdir("../input"))

# Any results you write to the current directory are saved as output.

import math
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.ensemble import RandomForestClassifier

['y_train.csv', 'sample_submission.csv', 'X_test.csv', 'X_train.csv']


**Importing Data Set**

In [56]:
X_train=pd.read_csv('../input/X_train.csv')
y_train=pd.read_csv('../input/y_train.csv')
X_test=pd.read_csv('../input/X_test.csv')
y_test=pd.read_csv('../input/sample_submission.csv')

In [57]:
X_train.describe()

,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
count,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000,487680.000000
mean,1904.500000,63.500000,-0.018050,0.075062,0.012458,-0.003804,0.000178,0.008338,-0.019184,0.129281,2.886468,-9.364886
std,1099.853353,36.949327,0.685696,0.708226,0.105972,0.104299,0.117764,0.088677,0.229153,1.870600,2.140067,2.845341
min,0.000000,0.000000,-0.989100,-0.989650,-0.162830,-0.156620,-2.371000,-0.927860,-1.268800,-36.067000,-121.490000,-75.386000
25%,952.000000,31.750000,-0.705120,-0.688980,-0.089466,-0.106060,-0.040752,-0.033191,-0.090743,-0.530833,1.957900,-10.193000
50%,1904.500000,63.500000,-0.105960,0.237855,0.031949,-0.018704,0.000084,0.005412,-0.005335,0.124980,2.879600,-9.365300
75%,2857.000000,95.250000,0.651803,0.809550,0.122870,0.097215,0.040527,0.048068,0.064604,0.792263,3.798800,-8.522700
max,3809.000000,127.000000,0.989100,0.988980,0.155710,0.154770,2.282200,1.079100,1.387300,36.797000,73.008000,65.839000


In [58]:
y_train.describe()

,series_id,group_id
count,3810.000000,3810.000000
mean,1904.500000,37.601312
std,1099.996591,20.982743
min,0.000000,0.000000
25%,952.250000,19.000000
50%,1904.500000,39.000000
75%,2856.750000,55.000000
max,3809.000000,72.000000


In [59]:
X_test.describe()

,series_id,measurement_number,orientation_X,orientation_Y,orientation_Z,orientation_W,angular_velocity_X,angular_velocity_Y,angular_velocity_Z,linear_acceleration_X,linear_acceleration_Y,linear_acceleration_Z
count,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000,488448.000000
mean,1907.500000,63.500000,0.031996,0.120651,0.018735,0.004024,0.000401,0.013669,-0.036101,0.125433,2.906588,-9.359811
std,1101.585403,36.949327,0.671977,0.714522,0.108481,0.100957,0.114485,0.089440,0.233791,1.832639,2.094727,2.802670
min,0.000000,0.000000,-0.989720,-0.989810,-0.154680,-0.156270,-2.403300,-1.193000,-1.458200,-35.040000,-107.480000,-83.173000
25%,953.750000,31.750000,-0.648130,-0.744503,-0.112660,-0.098435,-0.037396,-0.029865,-0.089842,-0.486233,2.052100,-10.125000
50%,1907.500000,63.500000,0.132910,0.397860,0.057271,0.022654,0.001107,0.007276,-0.008911,0.112490,2.900300,-9.361700
75%,2861.250000,95.250000,0.575270,0.803600,0.124770,0.083706,0.037827,0.051112,0.057876,0.727250,3.758000,-8.582075
max,3815.000000,127.000000,0.989320,0.988940,0.154250,0.152510,3.396600,1.150700,0.960280,36.628000,55.624000,59.633000


In [60]:
y_test.describe()

,series_id
count,3816.000000
mean,1907.500000
std,1101.728642
min,0.000000
25%,953.750000
50%,1907.500000
75%,2861.250000
max,3815.000000


In [61]:
def quaternion_to_euler(x, y, z, w):
    import math
    t0 = +2.0 * (w * x + y * z)
    t1 = +1.0 - 2.0 * (x * x + y * y)
    X = math.atan2(t0, t1)

    t2 = +2.0 * (w * y - z * x)
    t2 = +1.0 if t2 > +1.0 else t2
    t2 = -1.0 if t2 < -1.0 else t2
    Y = math.asin(t2)

    t3 = +2.0 * (w * z + x * y)
    t4 = +1.0 - 2.0 * (y * y + z * z)
    Z = math.atan2(t3, t4)

    return X, Y, Z

In [62]:
def perform_euler_factors_calculation(df):
    df['total_angular_velocity'] = np.sqrt(np.square(df['angular_velocity_X']) + np.square(df['angular_velocity_Y']) + np.square(df['angular_velocity_Z']))
    df['total_linear_acceleration'] = np.sqrt(np.square(df['linear_acceleration_X']) + np.square(df['linear_acceleration_Y']) + np.square(df['linear_acceleration_Z']))
    df['total_xyz'] = np.sqrt(np.square(df['orientation_X']) + np.square(df['orientation_Y']) +
                              np.square(df['orientation_Z']))
    df['acc_vs_vel'] = df['total_linear_acceleration'] / df['total_angular_velocity']
    
    x, y, z, w = df['orientation_X'].tolist(), df['orientation_Y'].tolist(), df['orientation_Z'].tolist(), df['orientation_W'].tolist()
    nx, ny, nz = [], [], []
    for i in range(len(x)):
        xx, yy, zz = quaternion_to_euler(x[i], y[i], z[i], w[i])
        nx.append(xx)
        ny.append(yy)
        nz.append(zz)
    
    df['euler_x'] = nx
    df['euler_y'] = ny
    df['euler_z'] = nz
    
    df['total_angle'] = np.sqrt(np.square(df['euler_x']) + np.square(df['euler_y']) + np.square(df['euler_z']))
    df['angle_vs_acc'] = df['total_angle'] / df['total_linear_acceleration']
    df['angle_vs_vel'] = df['total_angle'] / df['total_angular_velocity']
    return df

In [63]:
def perform_feature_engineering(df):
    df_out = pd.DataFrame()
    
    for col in df.columns:
        if col in ['row_id', 'series_id', 'measurement_number']:
            continue
        df_out[col + '_mean'] = df.groupby(['series_id'])[col].mean()
        df_out[col + '_min'] = df.groupby(['series_id'])[col].min()
        df_out[col + '_max'] = df.groupby(['series_id'])[col].max()
        df_out[col + '_std'] = df.groupby(['series_id'])[col].std()
        df_out[col + '_mad'] = df.groupby(['series_id'])[col].mad()
        df_out[col + '_med'] = df.groupby(['series_id'])[col].median()
        df_out[col + '_skew'] = df.groupby(['series_id'])[col].skew()
        df_out[col + '_range'] = df_out[col + '_max'] - df_out[col + '_min']
        df_out[col + '_max_to_min'] = df_out[col + '_max'] / df_out[col + '_min']
        df_out[col + '_mean_abs_change'] = df.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(np.diff(x))))
        df_out[col + '_mean_change_of_abs_change'] = df.groupby('series_id')[col].apply(lambda x: np.mean(np.diff(np.abs(np.diff(x)))))
        df_out[col + '_abs_max'] = df.groupby('series_id')[col].apply(lambda x: np.max(np.abs(x)))
        df_out[col + '_abs_min'] = df.groupby('series_id')[col].apply(lambda x: np.min(np.abs(x)))
        df_out[col + '_abs_mean'] = df.groupby('series_id')[col].apply(lambda x: np.mean(np.abs(x)))
        df_out[col + '_abs_std'] = df.groupby('series_id')[col].apply(lambda x: np.std(np.abs(x)))
        df_out[col + '_abs_avg'] = (df_out[col + '_abs_min'] + df_out[col + '_abs_max'])/2
        df_out[col + '_abs_range'] = df_out[col + '_abs_max'] - df_out[col + '_abs_min']

    return df_out

In [64]:
X_train = perform_euler_factors_calculation(X_train)

In [65]:
X_test = perform_euler_factors_calculation(X_test)

In [66]:
X_train.shape, X_test.shape

((487680, 23), (488448, 23))

In [67]:
X_train = perform_feature_engineering(X_train[X_train.columns.values[:13]])

In [68]:
X_test = perform_feature_engineering(X_test[X_test.columns.values[:13]])

In [69]:
print("Train X: {}\nTrain y: {}\nTest X: {}".format(X_train.shape, y_train.shape, X_test.shape))

Train X: (3810, 170)
Train y: (3810, 3)
Test X: (3816, 170)


In [70]:
le = LabelEncoder()
y_train['surface'] = le.fit_transform(y_train['surface'])

In [71]:
scaler = StandardScaler()
X_train_scaled = pd.DataFrame(scaler.fit_transform(X_train))
X_test_scaled = pd.DataFrame(scaler.transform(X_test))

In [72]:
folds = StratifiedKFold(n_splits=49, shuffle=True, random_state=41)

In [73]:
folds

StratifiedKFold(n_splits=49, random_state=41, shuffle=True)

In [74]:
sub_preds_rf = np.zeros((X_test_scaled.shape[0], 9))
oof_preds_rf = np.zeros((X_train_scaled.shape[0]))
score = 0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_scaled, y_train['surface'])):
    clf =  RandomForestClassifier(n_estimators = 500, n_jobs = -1)
    clf.fit(X_train_scaled.iloc[trn_idx], y_train['surface'][trn_idx])
    oof_preds_rf[val_idx] = clf.predict(X_train_scaled.iloc[val_idx])
    sub_preds_rf += clf.predict_proba(X_test_scaled) / folds.n_splits
    score += clf.score(X_train_scaled.iloc[val_idx], y_train['surface'][val_idx])
    print('Fold: {} score: {}'.format(fold_,clf.score(X_train_scaled.iloc[val_idx], y_train['surface'][val_idx])))
print('Avg Accuracy', score / folds.n_splits)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 21 members, which is too few. The minimum number of members in any class cannot be less than n_splits=49.
  % (min_groups, self.n_splits)), Warning)


Fold: 0 score: 0.9012345679012346
Fold: 1 score: 0.8395061728395061
Fold: 2 score: 0.8780487804878049
Fold: 3 score: 0.9
Fold: 4 score: 0.8375
Fold: 5 score: 0.8641975308641975
Fold: 6 score: 0.8641975308641975
Fold: 7 score: 0.8765432098765432
Fold: 8 score: 0.8765432098765432
Fold: 9 score: 0.925
Fold: 10 score: 0.8518518518518519
Fold: 11 score: 0.8271604938271605
Fold: 12 score: 0.9259259259259259
Fold: 13 score: 0.8625
Fold: 14 score: 0.8987341772151899
Fold: 15 score: 0.8354430379746836
Fold: 16 score: 0.8987341772151899
Fold: 17 score: 0.8860759493670886
Fold: 18 score: 0.875
Fold: 19 score: 0.8717948717948718
Fold: 20 score: 0.8717948717948718
Fold: 21 score: 0.8846153846153846
Fold: 22 score: 0.8961038961038961
Fold: 23 score: 0.8571428571428571
Fold: 24 score: 0.9090909090909091
Fold: 25 score: 0.8552631578947368
Fold: 26 score: 0.9090909090909091
Fold: 27 score: 0.868421052631579
Fold: 28 score: 0.8552631578947368
Fold: 29 score: 0.881578947368421
Fold: 30 score: 0.855263157

In [75]:
sub_preds_rf

array([[9.46938776e-03, 2.36734694e-03, 5.58775510e-02, ...,
        3.26530612e-04, 7.42857143e-02, 7.83673469e-03],
       [3.59102041e-01, 3.37959184e-01, 1.35836735e-01, ...,
        8.06122449e-02, 3.02857143e-02, 1.78775510e-02],
       [8.57142857e-04, 8.36734694e-03, 2.10693878e-01, ...,
        1.63265306e-04, 7.51510204e-01, 2.55918367e-02],
       ...,
       [1.67346939e-03, 3.09346939e-01, 4.51428571e-02, ...,
        1.55102041e-03, 5.54693878e-02, 1.62040816e-02],
       [3.01469388e-01, 3.53306122e-01, 3.38775510e-03, ...,
        6.12244898e-04, 2.13877551e-02, 3.10979592e-01],
       [5.73061224e-02, 1.69387755e-02, 2.22448980e-02, ...,
        1.02040816e-02, 3.06122449e-03, 8.34693878e-01]])

In [76]:
from xgboost import XGBClassifier
sub_preds_xgboost = np.zeros((X_test_scaled.shape[0], 9))
oof_preds_xgboost = np.zeros((X_train_scaled.shape[0]))
score = 0
for fold_, (trn_idx, val_idx) in enumerate(folds.split(X_train_scaled, y_train['surface'])):
    xgb_clf =  XGBClassifier(n_jobs = -1)
    xgb_clf.fit(X_train_scaled.iloc[trn_idx], y_train['surface'][trn_idx])
    oof_preds_xgboost[val_idx] = xgb_clf.predict(X_train_scaled.iloc[val_idx])
    sub_preds_xgboost += xgb_clf.predict_proba(X_test_scaled) / folds.n_splits
    score += xgb_clf.score(X_train_scaled.iloc[val_idx], y_train['surface'][val_idx])
    print('Fold: {} score: {}'.format(fold_,xgb_clf.score(X_train_scaled.iloc[val_idx], y_train['surface'][val_idx])))
print('Avg Accuracy', score / folds.n_splits)

/opt/conda/lib/python3.6/site-packages/sklearn/model_selection/_split.py:652: Warning: The least populated class in y has only 21 members, which is too few. The minimum number of members in any class cannot be less than n_splits=49.
  % (min_groups, self.n_splits)), Warning)


Fold: 0 score: 0.8765432098765432
Fold: 1 score: 0.8518518518518519
Fold: 2 score: 0.8414634146341463
Fold: 3 score: 0.875
Fold: 4 score: 0.85
Fold: 5 score: 0.8148148148148148
Fold: 6 score: 0.8641975308641975
Fold: 7 score: 0.7777777777777778
Fold: 8 score: 0.8271604938271605
Fold: 9 score: 0.8875
Fold: 10 score: 0.8765432098765432
Fold: 11 score: 0.7777777777777778
Fold: 12 score: 0.8765432098765432
Fold: 13 score: 0.7625
Fold: 14 score: 0.810126582278481
Fold: 15 score: 0.7974683544303798
Fold: 16 score: 0.8354430379746836
Fold: 17 score: 0.9240506329113924
Fold: 18 score: 0.8625
Fold: 19 score: 0.8333333333333334
Fold: 20 score: 0.8076923076923077
Fold: 21 score: 0.8333333333333334
Fold: 22 score: 0.8311688311688312
Fold: 23 score: 0.8181818181818182
Fold: 24 score: 0.8181818181818182
Fold: 25 score: 0.8026315789473685
Fold: 26 score: 0.8831168831168831
Fold: 27 score: 0.881578947368421
Fold: 28 score: 0.8026315789473685
Fold: 29 score: 0.8552631578947368
Fold: 30 score: 0.7894736

In [77]:
sub_preds_xgboost

array([[0.02508902, 0.26590877, 0.16870091, ..., 0.0017263 , 0.24219773,
        0.01253445],
       [0.68969683, 0.10057426, 0.01942149, ..., 0.01115817, 0.00993351,
        0.03361229],
       [0.00804054, 0.02855867, 0.08041759, ..., 0.00253336, 0.83341374,
        0.03058238],
       ...,
       [0.00907101, 0.58076299, 0.13718667, ..., 0.00677822, 0.02196682,
        0.15447385],
       [0.08408147, 0.52404486, 0.00983969, ..., 0.00325243, 0.03840813,
        0.3182782 ],
       [0.0241584 , 0.00268167, 0.00534552, ..., 0.00105918, 0.00719547,
        0.94572534]])

In [78]:
y_test_pred_final = np.array(sub_preds_rf.argmax(axis=1)*0.5+sub_preds_xgboost.argmax(axis=1)*0.5)

In [79]:
y_test_pred_final.shape

(3816,)

In [80]:
y_test_pred_final = np.array([int(val) for val in y_test_pred_final])

In [83]:
y_test_pred_final

array([4, 0, 7, ..., 2, 1, 8])

In [84]:
y_test['surface'] = le.inverse_transform(y_test_pred_final)
y_test.to_csv('submission_18.csv', index=False)

In [85]:
y_test

,series_id,surface
0,0,hard_tiles_large_space
1,1,carpet
2,2,tiled
3,3,hard_tiles
4,4,soft_tiles
5,5,concrete
6,6,soft_pvc
7,7,concrete
8,8,wood
9,9,wood
